In [14]:
import os
import logging
import gradio as gr
import pandas as pd
import numpy as np
from openai import OpenAI
from gtts import gTTS
import tempfile
from typing import Tuple, Dict, Any

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Initialize OpenAI client
client = OpenAI(api_key=("Your_OpenAI_API"))

# Constants
MAX_ROWS = 1000
MAX_COLS = 50   
VALID_FILE_TYPES = [".csv", ".xlsx", ".xls"]
SUPPORTED_AUDIO_FORMATS = ["wav", "mp3", "ogg"]

class DataAnalyzer:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        
    def validate_file(self, file_path: str) -> bool:
        if not any(file_path.endswith(ext) for ext in VALID_FILE_TYPES):
            raise ValueError("Unsupported file format. Please upload CSV or Excel file.")
        return True

    def read_data(self, file_path: str) -> pd.DataFrame:
        try:
            self.validate_file(file_path)
            if file_path.endswith(".csv"):
                df = pd.read_csv(file_path, encoding_errors='replace')
            else:
                df = pd.read_excel(file_path)
            
            df = df.dropna(how='all').dropna(how='all', axis=1)
            return df.iloc[:MAX_ROWS, :MAX_COLS]
        except Exception as e:
            self.logger.error(f"Error reading file: {str(e)}")
            raise

    def generate_summary(self, df: pd.DataFrame) -> Dict[str, Any]:
        summary = {
            "data_types": df.dtypes.astype(str).to_dict(),
            "missing_values": df.isnull().sum().to_dict(),
            "numeric_summary": df.describe().to_dict() if df.select_dtypes(include=np.number).shape[1] > 0 else {},
            "categorical_summary": {col: df[col].value_counts().to_dict() 
                                   for col in df.select_dtypes(include='object').columns}
        }
        return summary

class AIService:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
    
    def get_ai_response(self, system_prompt: str, user_prompt: str) -> str:
        try:
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.3,
                max_tokens=1500
            )
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"OpenAI API Error: {str(e)}")
            return f"AI Service Error: {str(e)}"
    
    def generate_insights(self, df: pd.DataFrame, prompt: str) -> str:
        system_prompt = f"You are a senior data scientist. Analyze this data and provide insights based on: {prompt}"
        data_preview = f"Data Columns: {list(df.columns)}\nSample Data:\n{df.head(3).to_string()}"
        return self.get_ai_response(system_prompt, data_preview + prompt)

class AudioService:
    @staticmethod
    def text_to_speech(text: str) -> Tuple[str, str]:
        try:
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as fp:
                tts = gTTS(text=text, lang='en', slow=False)
                tts.save(fp.name)
                return fp.name, None
        except Exception as e:
            logging.error(f"TTS Error: {str(e)}")
            return None, str(e)
    
    @staticmethod
    def speech_to_text(audio_path: str) -> Tuple[str, str]:
        try:
            with open(audio_path, "rb") as audio_file:
                transcript = client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file
                )
                return transcript.text, None
        except Exception as e:
            logging.error(f"STT Error: {str(e)}")
            return None, str(e)

# Initialize services
data_analyzer = DataAnalyzer()
ai_service = AIService()
audio_service = AudioService()

def process_inputs(file_path: str, prompt: str, audio_input: str) -> Tuple:
    try:
        if audio_input:
            transcript, error = audio_service.speech_to_text(audio_input)
            if error:
                raise ValueError(f"Speech recognition error: {error}")
            prompt = f"{prompt}\n[Voice Input]: {transcript}" if prompt else transcript

        df = data_analyzer.read_data(file_path)
        insights = ai_service.generate_insights(df, prompt)
        eda_report = ai_service.get_ai_response(
            system_prompt=f"Provide an EDA report based on {prompt}",
            user_prompt=f"Data Summary:\n{data_analyzer.generate_summary(df)}"
        )
        insight_audio, _ = audio_service.text_to_speech(insights)
        eda_audio, _ = audio_service.text_to_speech(eda_report)
        return insights, eda_report, insight_audio, eda_audio
    except Exception as e:
        logging.error(f"Processing Error: {str(e)}")
        return str(e), str(e), None, None

with gr.Blocks(title="AI Data Analysis Suite") as app:
    gr.Markdown("# AI Data Analysis Suite 🧠")
    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Data", file_types=VALID_FILE_TYPES)
            prompt_input = gr.Textbox(label="Enter Prompt")
            audio_input = gr.Audio(label="Or Speak Your Prompt", sources=["microphone"], type="filepath")
            process_btn = gr.Button("Analyze")
        with gr.Column():
            insights_output = gr.Textbox(label="AI Insights")
            insight_audio = gr.Audio(label="AI Audio Insights")
            eda_output = gr.Textbox(label="EDA Report")
            eda_audio = gr.Audio(label="EDA Audio Report")
    process_btn.click(fn=process_inputs, inputs=[file_input, prompt_input, audio_input], outputs=[insights_output, eda_output, insight_audio, eda_audio])
if __name__ == "__main__":
    app.launch(share=True)


2025-02-19 09:47:40,382 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-02-19 09:47:40,396 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7861


2025-02-19 09:47:41,245 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-02-19 09:47:41,861 - httpx - INFO - HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://e08b76066065ede033.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-02-19 09:47:44,996 - httpx - INFO - HTTP Request: HEAD https://e08b76066065ede033.gradio.live "HTTP/1.1 200 OK"
